In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from itertools import combinations_with_replacement as combs_r
from itertools import combinations as combs

####                    OPTIMIZTED GRADIENT DESCENT FUNCTION                   ####
def gradientDescent(X,y,theta, num_iters, alpha):
    m = X.shape[0]
    costs = np.zeros((num_iters,1));
    thetas = np.zeros((num_iters,theta.size));
    
    for i in range(num_iters):
        temp_thetas = (theta - alpha * ((np.dot(X.transpose(),(np.dot(X,theta)-y)))/m))

        theta = temp_thetas
        thetas[i] = temp_thetas.reshape(1,temp_thetas.size)
        costs[i] = calculateCost(X,y,theta)
        
        # plotting costs against the number of iterations
        # to check if gradient descent is working properly
        fig = plt.figure(figsize=(12,6))
        plt.title("Costs against number of iterations")
        plt.ylabel("Costs")
        plt.xlabel("Number of iterations")
        plt.plot(costs)
        plt.show()

        
    return {"hypothesis_theta":theta, "costs": costs, "thetas":thetas}






####                    OPTIMIZTED FEATURE NORMALIZATION FUNCTION                   ####
def featureNormalize(features):
    #Getting necessary variables
    means = features.mean(axis = 0)
    stds = features.std(axis = 0)

    featuresNorm = (features-means)/stds
            
    
    return {"normalized_features":featuresNorm, "means":means, "stds":stds}




####                    OPTIMIZTED FEATURE ADDITION FUNCTION                  ####

def addFeatures(X, num_combinations):
    if(num_combinations == 1 or num_combinations == 0):
        return X
    
    new_X = X
    num_features = X.shape[1]
    num_training_examples = X.shape[0]
    new_features = []
    #finding the number of permutations of the features
   
    
    for c in range(2, num_combinations+1):
        feature_combinations = combs_r(range(num_features),c)
        for feature_combination in feature_combinations:
            accumulator = 1
            for i in feature_combination:
                accumulator *= new_X[:, i]

            new_X = np.concatenate([new_X, accumulator.reshape(num_training_examples,1)], axis=1)
        
    return new_X





####                    NORMAL EQUATION FUNCTION                  ####
def normalEquation(X,y):
    X_t = X.transpose();
    hypothesis_theta = np.dot(np.linalg.pinv(np.dot(X_t,X)),np.dot(X_t,y))
    return hypothesis_theta





####                    REGULARIZED NORMAL EQUATION FUNCTION                  ####
def normalEquationReg(X,y,l):
    X_t = X.transpose();
    X_Xt = np.dot(X_t,X)
    identity = np.eye(X_Xt.shape[0],X_Xt.shape[1])
    identity[0,0] = 0
    hypothesis_theta = np.dot(np.linalg.pinv(X_Xt + l*(identity)),np.dot(X_t,y))
    return hypothesis_theta







####                    OPTIMIZED FUNCTION FOR CROSS VALIDATION                  ####
def crossValidate(X,y, X_cv, y_cv, num_degrees, title, ylabel):
    #setting up variable to hold final costs and degrees arrays
    costs = np.zeros(num_degrees)
    costs_cv = np.zeros(num_degrees)
    degrees = np.zeros(num_degrees)
    
    ##Testing for each of degrees in range num_degrees
    for i in range(1, num_degrees+1):
        temp_X = addFeatures(temp_X, i)
        temp_X = np.insert(temp_X, 0, 1, axis=1)
        temp_X_cv = addFeatures(temp_X_cv, i)
        temp_X_cv = np.insert(temp_X_cv, 0, 1, axis=1)
        
        theta = normalEquation(temp_X, y)
        costs[i-1] = calculateCost(temp_X, y, theta)
        costs_cv[i-1] = calculateCost(temp_X_cv, y_cv, theta)
        degrees[i-1] = i
        
    ##plotting the costs against degrees for both CV data and Training data
    fig = plt.figure(figsize=(12,6))
    plt.title(title)
    plt.xlabel("Degrees")
    plt.ylabel(ylabel)
    plt.plot(degrees,costs, label="Training data")
    plt.plot(degrees,costs_cv, label="Cv data")
    plt.legend()
    plt.show()
    
    ##Obtaining and returning the degree that produced the minimum cost CV data
    min_cost = min(costs_cv)
    min_degree = np.array(np.where(costs_cv == min_cost))[0][0] + 1
    
    return min_degree






####                    OPTIMIZED FUNCTION FOR FINDING OPTIMAL LAMBDA                  ####
## Function for finding optiaml lambda
def optimalLambda(X, y, X_cv, y_cv, lambdas, degree, title, ylabel):
    size_lambdas = lambdas.size
    costs = np.zeros(size_lambdas)
    costs_cv = np.zeros(size_lambdas)
    temp_X = addFeatures(X,degree)
    temp_X_cv = addFeatures(X_cv,degree)
    temp_X = np.insert(temp_X,0,1,axis=1)
    temp_X_cv = np.insert(temp_X_cv,0,1,axis=1)
    
    for i in range(size_lambdas):
        theta = normalEquationReg(temp_X,y,lambdas[i])
        costs[i] = calculateCost(temp_X,y,theta)
        costs_cv[i] = calculateCost(temp_X_cv,y_cv,theta)
        
    ##plotting the costs against lambdas for both CV data and Training data
    fig = plt.figure(figsize=(12,6))
    plt.title(title)
    plt.xlabel("Lambdas")
    plt.ylabel(ylabel)
    plt.plot(lambdas,costs, label="Training data")
    plt.plot(lambdas,costs_cv, label="Cv data")
    plt.legend()
    plt.show()
    
    ##Obtaining and returning the degree that produced the minimum cost CV data
    min_cost = min(costs_cv)
    min_lambda = lambdas[np.array(np.where(costs_cv == min_cost))[0][0]]
    return min_lambda
    
    
    
    
    



####                    FUNCTION FOR SEPARATING DATA INTO TRAINING, CROSS VALIDATION AND TEST                  ####
### -----NB: the data also returns the training data size

def trs_cvs_ts(X):
    
    ## !!! HAVE TO IMPORT math MODULE BEFORE RUNNING THIS FUNCTION
    
    num_training_egs = X.shape[0]
    training_data_size = 0.6 * num_training_egs
    cvs_ts_size = 0.2 * num_training_egs
    
    tr = 0
    cv = 0
    
    
    data = {}
    
#     #checking if flooring all would equal the number of training egs
#     if (int(training_data_size) + 2 * math.ceil(cvs_ts_size)) == num_training_egs:
#         tr = int(training_data_size)
#         cv = tr + math.ceil(cvs_ts_size)
#         ts = tr + cv + math.ceil(cvs_ts_size)
#     elif (math.ceil(training_data) + 2 * int(cvs_ts_size)) == num_training_egs:
#         tr = math.ceil(training_data_size)
#         cv = tr + int(cvs_ts_size)
#         ts = tr + cv + int(cvs_ts_size)
#     else:
#         tr = int(training_data_size)
#         cv = tr + int(cvs_ts_size)
#         ts = tr + cv + int(cvs_ts_size)
        
    data["trs"] = X[0:tr, :]
    data["cvs"] = X[tr:tr+cv, :]
    data["ts"] = X[tr+cv:0, :]
    data["trs_size"] = tr
    data["cvs_size"]
    
    return data




####                   OPTIMIZED FUNCTION FOR CHECKING BEST SIZE AND COMBINATIONS OF AVAILABLE FEATURES                 ####
def bestParams(X,y):
    features_size = X.shape[1]
    features  = range(features_size)
    diff_combs = []
    costs = []
    
    for i in range(1,features_size+1):
        c = cc(features,i)
        for combo in c:
            diff_combs.append(list(combo))
    
    for combo in diff_combs:
        test_features = X[:,combo]
        test_features = np.insert(test_features,0,1,axis=1)
    
        trcvts = trs_cvs_ts(test_features)
        X_tr = trcvts["trs"]
        X_cv = trcvts["cvs"]
        X_t = trcvts["ts"]
        trs_size = trcvts["trs_size"]
        cvs_size = trcvts["cvs_size"]
        last_index = trcvts["last_index"]
        
        y_tr = y[0:trs_size, :]
        y_cv = y[trs_size:cvs_size, :]
        y_t = y[cvs_size:last_index,:]
        
        theta = normalEquation(X_tr, y_tr)
        
        costs.append(calculateCost(X_cv, y_cv, theta))

        
    fig = plt.figure(figsize=(12,6))
    plt.title("Test different sizes and combinations of features")
    plt.xlabel("Index of combinattion")
    plt.ylabel("Costs")
    plt.plot(costs, "rx")
    plt.show()
    
    result = {}
    min_cost = min(costs)
    min_combo = diff_combs[costs.index(min(costs))]
    
    result["min_cost"] = min_cost
    result["min_combo"] = min_combo
    
    return result
    